In [2]:
import rpyc

class RemoteSCPI(object):
 
    def __init__(self, host, port):
        self._conn = rpyc.connect(host, port)
        self._inst = self._conn.root.instrument()
 
    def write(self, command):
        self._inst.write(command);
 
    def read(self):
        return self._inst.read()

    def ask(self, command):
        return self._inst.ask(command)
    
conn = RemoteSCPI('192.168.1.107', 18861)

In [39]:
class SCPI:
    def __init__(self, instr):
        self._instr = instr
        
    def scpi_write(self, cmd, *args):
        """ Execute a SCPI command
        for example :
        inst.scpi_write('FREQ', 1234)
        """
        arg_str = ','.join([repr(arg) for arg in args])
        self._instr.write(f'{cmd} {arg_str}')

    def scpi_ask(self, cmd):
        cmd = cmd if cmd.endswith('?') else cmd + '?'
        return self._instr.ask(cmd)

    def scpi_ask_for_float(self, cmd):
        """Ask and convert to float"""
        return float(self.scpi_ask(cmd))

    def get_idn(self):
        return tuple(self.scpi_ask('*IDN?').split(','))

        
    @property
    def idn(self):
        return self.get_idn()
    
    def get_manufacturer(self):
        return self.get_idn()[0]

    
class Scope():
    @property
    def channel1scale(self):
        return self.get_channel_scale(1)
    
    @channel1scale.setter
    def channel1scale(self, value):
        return self.set_channel_scale(1, value)

    @property
    def channel1(self):
        return Channel(self, 1)

    @property
    def channel(self):
        return Channels(self)

    
class TektronixScope(SCPI, Scope):
    
    def _create_channel_command(self, chan_number, cmd):
        assert chan_number in [1, 2, 3, 4]
        return f'CH{chan_number}:{cmd}'        
    
    def get_channel_scale(self, chan_number):
        cmd = self._create_channel_command(chan_number, 'SCA?')
        return self.scpi_ask_for_float(cmd)

    def set_channel_scale(self, chan_number, value):
        self.scpi_write(self._create_channel_command(chan_number, 'SCA'), value)

        
    
class Channels():
    def __init__(self, scope):
        self.scope = scope
    
    def __getitem__(self, chan_number):
        return Channel(self.scope, chan_number)
    
    
class Channel():
    def __init__(self, scope, chan_number):
        self.scope = scope
        self.chan_number = chan_number
        
    @property
    def scale(self):
        return self.scope.get_channel_scale(self.chan_number)

    @scale.setter
    def scale(self, value):
        return self.scope.set_channel_scale(self.chan_number, value)
    
scope = TektronixScope(conn)

scope.idn
print(scope.channel1scale)
scope.channel1.scale = 0.1
print(scope.channel1.scale)


0.0152
0.1


In [42]:
b'#40004\n\x0e\x10\x18'

b'#40004\n\x0e\x10\x18'

In [52]:
s = b'\n\x0e\x10\xff'

In [53]:
import numpy as np

np.frombuffer(s, dtype='int8')

array([10, 14, 16, -1], dtype=int8)

In [37]:
scope.channel[1].scale

0.3

In [ ]:
(scope.channel1).scale

In [55]:
class FakeSCPI(object):
    _record = {'*IDN':"TEKTRONIX,DPO3014,C012048,CF:91.1CT FV:v2.16 "}
    def write(self, val):
        if ' ' in val:
            cmd, vals = val.split(' ')
            self._record[cmd] = vals
    def ask(self, val):
        assert val[-1]=='?'
        out = self._record.get(val[:-1], '')
        return out

In [57]:
conn = FakeSCPI()

scope = TektronixScope(conn)

scope.idn

scope.channel[1].scale = 10
scope.channel[1].scale

10.0